# Face recognizer using pre trained model

Below is the implementation of a face recognizer which can identify the face of the person showing on a web cam. We will apply transfer learning and use pre-trained weights of VGG Face model.

# Import the requried libraries
Here we are importing all the requried libraries.

In [1]:
from keras.models import Model, Sequential
from keras.layers import Input, Convolution2D, ZeroPadding2D, MaxPooling2D, Flatten, Dense, Dropout, Activation
from PIL import Image
import numpy as np
from keras.preprocessing.image import load_img, save_img, img_to_array
from keras.applications.imagenet_utils import preprocess_input
from keras.preprocessing import image
import matplotlib.pyplot as plt
import cv2
import os

Using TensorFlow backend.


# Contructing the neural network model
The model here constructed is based on VGG face model.It has 22 layers and 37 deep units.
![Vgg face model](images/vgg-face-model.png)
The structure of the VGG-Face model is demonstrated below.


The VGG-Face architure can be to be understood clear with the help of following image,
![Vgg face model](images/vgg-face-architecture.jpg)



In [2]:
model = Sequential()
model.add(ZeroPadding2D((1,1),input_shape=(224,224, 3)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(Convolution2D(4096, (7, 7), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(4096, (1, 1), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(2622, (1, 1)))
model.add(Flatten())
model.add(Activation('softmax'))

# Loading the model with pretrained weights



In [3]:



from keras.models import model_from_json

model.load_weights('D:/AI/Face Recognition/face_recognition_FILES/vgg_face_weights.h5')
# model.load_weights('C:/Users/Abhay/Desktop/Face-recognition-using-deep-learning-master/vgg_face_weights.h5')

## About <font color=blue>image_to_embedding</font> function        
When the model is loaded with pre trained weights, then we can create the **2622 dimensional embedding vectors** for all the face images stored in the "images" folder. **"image_to_embedding"** function pass an image to the VGG network to generate the embedding vector.

In [4]:
def image_to_embedding(image, model):
    #image = cv2.resize(image, (96, 96), interpolation=cv2.INTER_AREA) 
    image = cv2.resize(image, (224, 224)) 
    img = image[...,::-1]
    img = np.around(np.transpose(img, (0,1,2))/255.0, decimals=12)
    x_train = np.array([img])
    embedding = model.predict_on_batch(x_train)
    return embedding


## About <font color=blue>recognize_face</font> function
This function calculate similarity between the captured image and the images that are already been stored in the folder. It passes the image to the trained neural network to generate its embedding vector. Which is then compared with all the embedding vectors of the images stored by calculating L2 Euclidean distance and the cosine distance. 

Here I am using both the [Cosine Similarity](https://en.wikipedia.org/wiki/Cosine_similarity) and the [Euclidean distance](https://en.wikipedia.org/wiki/Euclidean_distance) to see for the match.Greater the Cosine similarity between two images then there is greater probability of having the image of same person. Smaller the Euclidean distance between two images then there is greater probability of having the image of same person. 

Here I am setting the thresholds for the Euclidean distance and Cosine Similarity are 0.17 and 0.70 (which can be adjusted).

In [5]:
def recognize_face(face_image, input_embeddings, model):   
    
    embedding = image_to_embedding(face_image, model)
    
    minimum_distance = 200
    minimum_distance1 = 0
    name = None
    
    # Loop over  names and encodings.
    for (input_name, input_embedding) in input_embeddings.items():
        
       
        euclidean_distance = np.linalg.norm(embedding-input_embedding) * 1000
        
        a = np.matmul(embedding,input_embedding.T)
        b = np.sqrt(np.matmul(embedding,embedding.T))
        c = np.sqrt(np.matmul(input_embedding,input_embedding.T))
        cosine_dist = a / ( b * c )
        
        #print(dist1)
        #print (cosine_dist)
        
      
        print('Euclidean distance from %s is %s' %(input_name, euclidean_distance))

        
        if euclidean_distance < minimum_distance and cosine_dist > minimum_distance1:
            minimum_distance = euclidean_distance
            minimum_distance1 = cosine_dist
            name = input_name[0]
            
    
    if minimum_distance < 0.17 and minimum_distance1 > 0.70:
        return str(name)
    else:
        return None
    

## About <font color=blue>create_input_image_embeddings</font> function
This function generates 512 dimensional image ebeddings of all the images stored in the "imagesv1" containg folders with person  names with images.We feed forwarding the images to a trained neural network. It creates a dictionary with key as the name of the face and value as embedding


In [6]:
import glob

def create_input_image_embeddings():      
    input_embeddings = {}

    for file in glob.glob("imagesv1/*"):
        for file1 in glob.glob(file+"/*"):
            person_name_image = os.path.splitext(os.path.basename(file1))[0]
            person_name = os.path.splitext(os.path.basename(file))[0]
#             print(person_name)
            image_file = cv2.imread(file1, 1)
            input_embeddings[person_name,person_name_image] = image_to_embedding(image_file, model)

    return input_embeddings

## About <font color=blue>recognize_faces_in_cam</font> function
This function capture image from the webcam, detect a face in it and crop the image to have a face only, which is then passed to recognize_face function. Here for the face detection we are using Haarcascade model.

In [7]:
def recognize_faces_in_cam(input_embeddings):
    

    cv2.namedWindow("Face Recognizer")
    vc = cv2.VideoCapture(0)
   

    font = cv2.FONT_HERSHEY_SIMPLEX
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    
    
    while vc.isOpened():
        _, frame = vc.read()
        img = frame
        height, width, channels = frame.shape

        
        
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)

        # Loop through all the faces detected 
        identities = []
        for (x, y, w, h) in faces:
            x1 = x
            y1 = y
            x2 = x+w
            y2 = y+h

           
            
            face_image = frame[max(0, y1):min(height, y2), max(0, x1):min(width, x2)]    
            identity = recognize_face(face_image, input_embeddings, model)
            
            

            if identity is not None:
                img = cv2.rectangle(frame,(x1, y1),(x2, y2),(255,255,255),2)
                cv2.putText(img, str(identity), (x1+5,y1-5), font, 1, (255,255,255), 2)
        
        key = cv2.waitKey(100)
        cv2.imshow("Face Recognizer", img)

        if key == 27: # exit on ESC
            break
    vc.release()
    cv2.destroyAllWindows()

## Capturing the face image
Following code captures 10 face images of the person. They all are stored in **"imagesv1"** folder.After every run this will create a new folder with name **"new user_(current date and time)"** (inorder to avoid duplication of the folder name).In this sub folder there will be 10 images of detected faces will be strored.You have to rename the folder name with the person name in order to get the correct name of person in output.You can also remove the images which are not much good from set of 10 images.

These images will be used for recognizing the the identity of the person using one shot learning.

In [21]:
cam = cv2.VideoCapture(0)

face_detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

#  #change - to get current time and date
from datetime import datetime
date_time = datetime.now().strftime('%Y%m%d%H%M%S')

#  #change create a new folder to save images
newpath =r'C:/Users/Abhay/Desktop/Face-recognition-using-deep-learning-master/imagesv1/newUser'+date_time 
image_path = r'imagesv1\newUser'+date_time
if not os.path.exists(newpath):
    os.makedirs(newpath)
    
    
count = 0
while(True):
    ret, img = cam.read()
    #gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_detector.detectMultiScale(img, 1.3, 5)
    for (x,y,w,h) in faces:
        x1 = x
        y1 = y
        x2 = x+w
        y2 = y+h
        cv2.rectangle(img, (x1,y1), (x2,y2), (255,255,255), 2)     
        count += 1
        # Save the captured image into the datasets folder
        cv2.imwrite(image_path+"/User_" + str(count) + ".jpg", img[y1:y2,x1:x2])
        cv2.imshow('image', img)
    k = cv2.waitKey(200) & 0xff # Press 'ESC' for exiting video
    if k == 27:
        break
    elif count >= 10: # Take 10 face sample and stop video
         break
cam.release()
cv2.destroyAllWindows()

In [8]:
input_embeddings = create_input_image_embeddings()
recognize_faces_in_cam(input_embeddings)


Euclidean distance from ('Abhi', 'abz') is 0.18194617587141693
Euclidean distance from ('Abhi', 'User_10') is 0.14620015281252563
Euclidean distance from ('Abhi', 'User_8') is 0.13905303785577416
Euclidean distance from ('Abhi', 'User_9') is 0.14496683434117585
Euclidean distance from ('Abhinav', 'User_4') is 0.282454100670293
Euclidean distance from ('Abhinav', 'User_5') is 0.2714337024372071
Euclidean distance from ('Abhinav', 'User_7') is 0.2534161030780524
Euclidean distance from ('Abhinav', 'User_8') is 0.2543556038290262
Euclidean distance from ('Abhinav', 'User_9') is 0.27103067259304225
Euclidean distance from ('Madhur', 'User_10') is 0.23197064001578838
Euclidean distance from ('Madhur', 'User_2') is 0.2373447932768613
Euclidean distance from ('Madhur', 'User_3') is 0.24341318930964917
Euclidean distance from ('Madhur', 'User_5') is 0.24849417968653142
Euclidean distance from ('Madhur', 'User_8') is 0.23996940581128
Euclidean distance from ('Madhur', 'User_9') is 0.24247993133

Euclidean distance from ('Abhi', 'abz') is 0.19784868345595896
Euclidean distance from ('Abhi', 'User_10') is 0.13702716387342662
Euclidean distance from ('Abhi', 'User_8') is 0.12740868260152638
Euclidean distance from ('Abhi', 'User_9') is 0.1352716062683612
Euclidean distance from ('Abhinav', 'User_4') is 0.29973822529427707
Euclidean distance from ('Abhinav', 'User_5') is 0.28820548322983086
Euclidean distance from ('Abhinav', 'User_7') is 0.2671506954357028
Euclidean distance from ('Abhinav', 'User_8') is 0.26713262195698917
Euclidean distance from ('Abhinav', 'User_9') is 0.28427602956071496
Euclidean distance from ('Madhur', 'User_10') is 0.22681205882690847
Euclidean distance from ('Madhur', 'User_2') is 0.2355988253839314
Euclidean distance from ('Madhur', 'User_3') is 0.24080296861939132
Euclidean distance from ('Madhur', 'User_5') is 0.2454601344652474
Euclidean distance from ('Madhur', 'User_8') is 0.2366663538850844
Euclidean distance from ('Madhur', 'User_9') is 0.2380217

Euclidean distance from ('Abhi', 'abz') is 0.19183638505637646
Euclidean distance from ('Abhi', 'User_10') is 0.1346459030173719
Euclidean distance from ('Abhi', 'User_8') is 0.12587073433678597
Euclidean distance from ('Abhi', 'User_9') is 0.13237570237834007
Euclidean distance from ('Abhinav', 'User_4') is 0.30398304807022214
Euclidean distance from ('Abhinav', 'User_5') is 0.293051271000877
Euclidean distance from ('Abhinav', 'User_7') is 0.27246761601418257
Euclidean distance from ('Abhinav', 'User_8') is 0.2716821909416467
Euclidean distance from ('Abhinav', 'User_9') is 0.29083460685797036
Euclidean distance from ('Madhur', 'User_10') is 0.2323135850019753
Euclidean distance from ('Madhur', 'User_2') is 0.24129720986820757
Euclidean distance from ('Madhur', 'User_3') is 0.24677437613718212
Euclidean distance from ('Madhur', 'User_5') is 0.25172342429868877
Euclidean distance from ('Madhur', 'User_8') is 0.24226079403888434
Euclidean distance from ('Madhur', 'User_9') is 0.2433743

Euclidean distance from ('Abhi', 'abz') is 0.18802225531544536
Euclidean distance from ('Abhi', 'User_10') is 0.15958689618855715
Euclidean distance from ('Abhi', 'User_8') is 0.1541800593258813
Euclidean distance from ('Abhi', 'User_9') is 0.159976800205186
Euclidean distance from ('Abhinav', 'User_4') is 0.28485446819104254
Euclidean distance from ('Abhinav', 'User_5') is 0.27561181923374534
Euclidean distance from ('Abhinav', 'User_7') is 0.2582968445494771
Euclidean distance from ('Abhinav', 'User_8') is 0.2598620776552707
Euclidean distance from ('Abhinav', 'User_9') is 0.27433232753537595
Euclidean distance from ('Madhur', 'User_10') is 0.22873847046867013
Euclidean distance from ('Madhur', 'User_2') is 0.23545724980067462
Euclidean distance from ('Madhur', 'User_3') is 0.24120791931636631
Euclidean distance from ('Madhur', 'User_5') is 0.24545702035538852
Euclidean distance from ('Madhur', 'User_8') is 0.23674278054386377
Euclidean distance from ('Madhur', 'User_9') is 0.2378884

Euclidean distance from ('Abhi', 'abz') is 0.23872932069934905
Euclidean distance from ('Abhi', 'User_10') is 0.19756221445277333
Euclidean distance from ('Abhi', 'User_8') is 0.1860458723967895
Euclidean distance from ('Abhi', 'User_9') is 0.19668086315505207
Euclidean distance from ('Abhinav', 'User_4') is 0.3237364289816469
Euclidean distance from ('Abhinav', 'User_5') is 0.31476354342885315
Euclidean distance from ('Abhinav', 'User_7') is 0.29582870774902403
Euclidean distance from ('Abhinav', 'User_8') is 0.2987357438541949
Euclidean distance from ('Abhinav', 'User_9') is 0.31007634242996573
Euclidean distance from ('Madhur', 'User_10') is 0.2527789620216936
Euclidean distance from ('Madhur', 'User_2') is 0.2608143840916455
Euclidean distance from ('Madhur', 'User_3') is 0.26671370142139494
Euclidean distance from ('Madhur', 'User_5') is 0.27167171356268227
Euclidean distance from ('Madhur', 'User_8') is 0.2604267792776227
Euclidean distance from ('Madhur', 'User_9') is 0.26179983

...

Euclidean distance from ('Abhi', 'abz') is 0.17956057854462415
Euclidean distance from ('Abhi', 'User_10') is 0.12468354543671012
Euclidean distance from ('Abhi', 'User_8') is 0.1198535246658139
Euclidean distance from ('Abhi', 'User_9') is 0.12313068145886064
Euclidean distance from ('Abhinav', 'User_4') is 0.29919727239757776
Euclidean distance from ('Abhinav', 'User_5') is 0.2881629625335336
Euclidean distance from ('Abhinav', 'User_7') is 0.2677410375326872
Euclidean distance from ('Abhinav', 'User_8') is 0.26782683562487364
Euclidean distance from ('Abhinav', 'User_9') is 0.2865251153707504
Euclidean distance from ('Madhur', 'User_10') is 0.22780094877816737
Euclidean distance from ('Madhur', 'User_2') is 0.2367392589803785
Euclidean distance from ('Madhur', 'User_3') is 0.24270253197755665
Euclidean distance from ('Madhur', 'User_5') is 0.24832756025716662
Euclidean distance from ('Madhur', 'User_8') is 0.2358301862841472
Euclidean distance from ('Madhur', 'User_9') is 0.23867296

Euclidean distance from ('Abhi', 'abz') is 0.16796762065496296
Euclidean distance from ('Abhi', 'User_10') is 0.11537561658769846
Euclidean distance from ('Abhi', 'User_8') is 0.11101817653980106
Euclidean distance from ('Abhi', 'User_9') is 0.11485532741062343
Euclidean distance from ('Abhinav', 'User_4') is 0.2870812895707786
Euclidean distance from ('Abhinav', 'User_5') is 0.2760631323326379
Euclidean distance from ('Abhinav', 'User_7') is 0.25677497615106404
Euclidean distance from ('Abhinav', 'User_8') is 0.2585271722637117
Euclidean distance from ('Abhinav', 'User_9') is 0.277759856544435
Euclidean distance from ('Madhur', 'User_10') is 0.21939186262898147
Euclidean distance from ('Madhur', 'User_2') is 0.22825485211797059
Euclidean distance from ('Madhur', 'User_3') is 0.23412579321302474
Euclidean distance from ('Madhur', 'User_5') is 0.2395498304395005
Euclidean distance from ('Madhur', 'User_8') is 0.22694059589412063
Euclidean distance from ('Madhur', 'User_9') is 0.23018344

Euclidean distance from ('Abhi', 'abz') is 0.17642036254983395
Euclidean distance from ('Abhi', 'User_10') is 0.1012806169455871
Euclidean distance from ('Abhi', 'User_8') is 0.0976602386799641
Euclidean distance from ('Abhi', 'User_9') is 0.09797612437978387
Euclidean distance from ('Abhinav', 'User_4') is 0.2823050017468631
Euclidean distance from ('Abhinav', 'User_5') is 0.27009955374524
Euclidean distance from ('Abhinav', 'User_7') is 0.24979031877592206
Euclidean distance from ('Abhinav', 'User_8') is 0.25028965319506824
Euclidean distance from ('Abhinav', 'User_9') is 0.2707162348087877
Euclidean distance from ('Madhur', 'User_10') is 0.21746075071860105
Euclidean distance from ('Madhur', 'User_2') is 0.22720685228705406
Euclidean distance from ('Madhur', 'User_3') is 0.23253264953382313
Euclidean distance from ('Madhur', 'User_5') is 0.23756333393976092
Euclidean distance from ('Madhur', 'User_8') is 0.22534684103447944
Euclidean distance from ('Madhur', 'User_9') is 0.228188480

Euclidean distance from ('Abhi', 'abz') is 0.17741729971021414
Euclidean distance from ('Abhi', 'User_10') is 0.12982146290596575
Euclidean distance from ('Abhi', 'User_8') is 0.12698235514108092
Euclidean distance from ('Abhi', 'User_9') is 0.13140120427124202
Euclidean distance from ('Abhinav', 'User_4') is 0.2951083588413894
Euclidean distance from ('Abhinav', 'User_5') is 0.2841420646291226
Euclidean distance from ('Abhinav', 'User_7') is 0.26407191762700677
Euclidean distance from ('Abhinav', 'User_8') is 0.26513347984291613
Euclidean distance from ('Abhinav', 'User_9') is 0.2814264444168657
Euclidean distance from ('Madhur', 'User_10') is 0.22973809973336756
Euclidean distance from ('Madhur', 'User_2') is 0.23950106697157025
Euclidean distance from ('Madhur', 'User_3') is 0.2446124271955341
Euclidean distance from ('Madhur', 'User_5') is 0.2486167650204152
Euclidean distance from ('Madhur', 'User_8') is 0.23674900876358151
Euclidean distance from ('Madhur', 'User_9') is 0.2391487

KeyboardInterrupt: 